In [22]:
import robotic as ry
import numpy as np

from time import sleep

import math
import sys


r1 = "r1"
r2 = "r2"
m1 = "m1"
m2 = "m2"
g1 = "g1"
g2 = "g2"

robots = [r1, r2]
m_objects = [m1, m2]
goals = [g1, g2]

goals_reverse = [g2, g1]

def komo_solver(robots, m_objects, goals):
    
    if not isinstance(robots, list):
        robots = [robots]
    if not isinstance(m_objects, list):
        m_objects = [m_objects]
    if not isinstance(goals, list):
        goals = [goals]
    
    komo = ry.KOMO(C, phases=2, slicesPerPhase=20, kOrder=2, enableCollisions=True)  
    komo.addControlObjective([], 0)
    komo.addControlObjective([], 2)
    
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)     
    
    for i in range(len(robots)):
        komo.addObjective([1, -1], ry.FS.distance, [robots[i], m_objects[i]], ry.OT.eq) 
        komo.addModeSwitch([1,-1], ry.SY.stable, [robots[i], m_objects[i]], True) 
        komo.addObjective([2] , ry.FS.positionDiff, [m_objects[i], goals[i]], ry.OT.eq) 

    ret = ry.NLP_Solver(komo.nlp(), verbose=4).solve() 
    
    return (komo, ret)

def komo_onestate_hold(robot, m_object):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=True)
    komo.addControlObjective([], 0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)


def komo_onestate_move(robot, m_object, goal):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder= 1, enableCollisions=True)
    komo.addModeSwitch([0,1], ry.SY.stable, [robot, m_object], True) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    komo.addObjective([1, 2] , ry.FS.positionDiff, [m_object, goal], ry.OT.eq) 
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)

    

def rrt_solver(k, start_state, end_state, trial_count):
    rrt = ry.PathFinder()
    rrt.setProblem(k, start_state, end_state)
    
    ret = rrt.solve()
    
        
    for i in range(trial_count):
        if ret.feasible:
            return ret
        ret = rrt.solve() 
        print(ret.feasible) 
    
    
    
    return ret


def rrt_viewer(C, ret, attach_state = None, attach = False):
    
    print("ikbngşbnşfgbnşjkfgnbjgfn")
    
    sleep(1)
    if not ret.feasible:
        return
    
    path = ret.x
    
    initial_state = C.getFrameState()
    
    if attach:
        
        C.setJointState(attach_state)
        C.attach(r1, m1)
        C.attach(r2, m2)
    
    
    for i in path:
        C.setJointState(i)
        C.view()
        sleep(0.3)
    
    C.setFrameState(initial_state)
    
    if attach:
        C.getFrame(m1).unLink()
        C.getFrame(m2).unLink()
        
def path_viewer(C, path, attach_index1, attach_index2):
    
    initial_state = C.getFrameState()
    
    C.view()

    sleep(2)
    
    for i in range(0, len(path)):
        
        
        C.setJointState(path[i])
        C.view()
        sleep(0.05)
        
        if i == attach_index1:
            C.attach(r1, m1)
        if i == attach_index2:
            C.attach(r2, m2)
            
    C.setFrameState(initial_state)
    
    C.getFrame(m1).unLink()
    C.getFrame(m2).unLink()
        
        
        

    
    

C = ry.Config()

maps = ["map1_simple_cross.g", "map2_tunnel.g", "map3_two_cave.g", "map4_one_cave.g", "map5_pioneer.g"]

C.addFile(f"maps/{maps[1]}")
C.view()











def path_mapper(path, final_size):
    original_path, hold_index = path
    original_path = np.array(original_path)
    path_len = len(original_path)

    # Sample evenly spaced indices, reserving one for final state and one for hold
    steps_to_sample = final_size - 2  # 1 for hold, 1 for final
    sample_indices = np.linspace(0, path_len - 2, steps_to_sample).astype(int)
    sampled_path = original_path[sample_indices]

    # Prepare hold and final states
    
    final_state = original_path[-1].reshape(1, -1)
    
    insert_pos = -1
    
    if hold_index > 0:
        hold_state = original_path[hold_index].reshape(1, -1)
        insert_pos = np.searchsorted(sample_indices, hold_index)
        with_hold = np.insert(sampled_path, insert_pos, hold_state, axis=0)
        final_result = np.vstack([with_hold, final_state])

    else:
        final_result = np.vstack([sampled_path, final_state])
            
    return final_result, insert_pos






    




In [23]:
initial_state = C.getJointState()

first_robot_hold, ret1 = komo_onestate_hold(r1, m1)  
second_robot_hold, ret2 = komo_onestate_hold(r2, m2) 

j1_hold = first_robot_hold.getPath()[0]  # birinci tobotun cargonun yanına geldiği state
j2_hold = second_robot_hold.getPath()[0]      # ikinci tobotun cargonun yanına geldiği state

first_robot_move, ret1 = komo_onestate_move(r1, m1, g2)
second_robot_move, ret2 = komo_onestate_move(r2, m2, g1)

j1_moved = first_robot_move.getPath()[0] # birinci robotun goal yanına geldiği state
j2_moved = second_robot_move.getPath()[0] # ikinci robotun goal yanına geldiği state

#combined_hold = np.concatenate((j1_moved[:3], j2_moved[-3:]))
#C.setJointState(combined_hold)
#C.view()

r1_hold_path = rrt_solver(C, initial_state, j1_hold, 3)
r1_move_path = rrt_solver(C, j1_hold, j1_moved, 3)
r1_final = (np.array([i[0:3] for i in np.concatenate((r1_hold_path.x, r1_move_path.x))]), len(r1_hold_path.x))


r2_hold_path = rrt_solver(C, initial_state, j2_hold, 3)
r2_move_path = rrt_solver(C, j2_hold, j2_moved, 3)
r2_final = (np.array([k[3:] for k in np.concatenate((r2_hold_path.x, r2_move_path.x))]), len(r2_hold_path.x))

r1_final = path_mapper(r1_final, 60)
r2_final = path_mapper(r2_final, 60)

final_path = np.hstack((r1_final[0], r2_final[0]))

#path_viewer(C, final_path, r1_final[1], r2_final[1])




In [24]:
# heuristic one -- checking for any collision


def collision_states(r1_path, r2_path):   # 0.8 seems to be working well
    
    state_indices = []
    
    for i in range(0, len(r1_path)):
        
        r1_loc = r1_path[i]
        r2_loc = r2_path[i]
    
        distance = math.sqrt(pow(abs(r1_loc[0] - r2_loc[0]), 2) + pow(abs(r1_loc[1] - r2_loc[1]), 2))
                    
        if distance < 1.1:
            state_indices.append(i)
            
    return state_indices



collision_indices = collision_states(r1_final[0], r2_final[0])

print(collision_indices)

if len(collision_indices) == 0:
    print("problem solved ....")
    sys.exit()
    
def find_another_path_in_collision(collision_states, r1_final, r2_final):
    
    r1_path = r1_final[0]
    r2_path = r2_final[0]
    
    temp = ry.Config()
    temp.addConfigurationCopy(C)
    
    sstt = np.concatenate((r1_final[0][r1_final[1]], r2_final[0][r2_final[1]]))
    temp.setJointState(sstt)
    
    temp.attach(r1, m1)
    temp.attach(r2, m2)
        
    rrt_again_initial = np.concatenate((r1_path[collision_states[0] - 3], r2_path[collision_states[0] - 3]))
    rrt_again_last = np.concatenate((r1_path[collision_states[-1] + 3], r2_path[collision_states[-1] + 3]))
    
    
    
    rrt_again = rrt_solver(temp, rrt_again_initial, rrt_again_last, 5)
    
    if rrt_again.feasible:
        return False
    
    current_path = path_mapper((rrt_again.x, -1), len(collision_indices) + 7)
    
    del temp
    return current_path


def manipulate_main(r1_final, r2_final, collision_indices, current_path):
    r1_path, r1_hold_idx = r1_final
    r2_path, r2_hold_idx = r2_final

    # Compute range to overwrite
    start = max(collision_indices[0] - 3, 0)
    end = min(collision_indices[-1] + 3, min(len(r1_path), len(r2_path), len(current_path[0]) + start + 1))  # assumes r1, r2 same length
    
    print("start: ", start)
    print("end:", end)
    print("len:", len(current_path[0]))

    # Replace values in r1 and r2 from current_path
    r1_path[start:end] = current_path[0][0:end - start, :3]   # First 3 values
    r2_path[start:end] = current_path[0][0:end - start, 3:]   # Last 3 values

    return (r1_path, r1_hold_idx), (r2_path, r2_hold_idx)


def make_wait(r1_final, r2_final, collision_indices):
    r1_path, r1_hold_idx = r1_final
    r2_path, r2_hold_idx = r2_final

    # How many steps to repeat
    wait_len = int(len(collision_indices) / 2)
    insert_idx = collision_indices[0] - 3

    # Get the state to repeat in r1
    wait_state_r1 = r1_path[insert_idx]
    repeated_r1 = np.repeat(wait_state_r1.reshape(1, -1), wait_len, axis=0)

    # Insert wait_state into r1_path at the collision start index
    r1_path = np.insert(r1_path, insert_idx, repeated_r1, axis=0)

    # Extend r2_path by repeating the last element wait_len times
    last_r2 = r2_path[-1]
    repeated_r2 = np.repeat(last_r2.reshape(1, -1), wait_len, axis=0)
    r2_path = np.concatenate((r2_path, repeated_r2), axis=0)

    return (r1_path, r1_hold_idx), (r2_path, r2_hold_idx)




newPath = find_another_path_in_collision(collision_indices, r1_final, r2_final)
newPath = False

if not newPath:
    
    print("wait wait wait ")
    r1_final, r2_final = make_wait(r1_final, r2_final, collision_indices)
    
    print(r1_final[0])
    
    
    newCollisiion = collision_states(r1_final[0], r2_final[0])
    
    if len(newCollisiion) == 0:
        print("problem solved via waiting")
        final_path = np.hstack((r1_final[0], r2_final[0]))
        
        path_viewer(C, final_path, r1_final[1], r2_final[1])
        
        sys.exit()
    

if newPath:
    r1_final, r2_final = manipulate_main(r1_final, r2_final, collision_indices, newPath)
    final_path = np.hstack((r1_final[0], r2_final[0]))

    sys.exit("problem solved")



    
            
        
        
        
    
    


[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]
wait wait wait 
[[ 4.00000000e+00 -4.00000000e+00  0.00000000e+00]
 [ 3.83836285e+00 -3.86798687e+00 -2.26085086e-03]
 [ 3.56085593e+00 -3.91827032e+00  1.19654226e-02]
 [ 3.37229793e+00 -3.97545704e+00 -3.43326366e-03]
 [ 3.14082539e+00 -4.00248632e+00 -2.48886574e-02]
 [ 3.00003193e+00 -3.99999974e+00 -1.12634237e-06]
 [ 3.00003193e+00 -3.99999974e+00 -1.12634237e-06]
 [ 3.01612855e+00 -3.79303083e+00  2.12281500e-02]
 [ 2.92694464e+00 -3.51376936e+00  2.76914552e-02]
 [ 2.74568869e+00 -3.31060229e+00  5.49183055e-03]
 [ 2.60588405e+00 -3.09108301e+00 -3.50133006e-03]
 [ 2.42267272e+00 -2.85366505e+00 -8.29179735e-03]
 [ 2.21691312e+00 -2.84865512e+00  1.63286711e-02]
 [ 2.05265090e+00 -2.80857516e+00 -2.37744380e-02]
 [ 1.82786497e+00 -2.82270831e+00  5.27981692e-02]
 [ 1.63992180e+00 -2.80645075e+00  9.25216338e-02]
 [ 1.40892270e+00 -2.82455591e+00  4.15496194e-02]
 [ 1.23837511e+00 -2.846

SystemExit: 

In [ ]:
path_viewer(C, final_path, r1_final[1], r2_final[1])